In [142]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import truncnorm

In [143]:
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/wheat-seeds.csv"
df = pd.read_csv(url, header=None)
df.columns = ['area', 
              'perimeter',
              'compactness',
              'length of kernel',
              'width of kernel',
              'asymmetry coefficient',
              'length of kernel groove',
             'target']

In [144]:
df

,area,perimeter,compactness,length of kernel,width of kernel,asymmetry coefficient,length of kernel groove,target
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1
...,...,...,...,...,...,...,...,...
205,12.19,13.20,0.8783,5.137,2.981,3.631,4.870,3
206,11.23,12.88,0.8511,5.140,2.795,4.325,5.003,3
207,13.20,13.66,0.8883,5.236,3.232,8.315,5.056,3
208,11.84,13.21,0.8521,5.175,2.836,3.598,5.044,3


In [145]:
y = df.iloc[:, -1]

In [146]:
X = df.drop('target', axis=1)

# 1. Initialize Network

In [156]:
def init_weights(X, n_neurons):
    n_inputs = int(X.shape[1])
    stddev = 2 / np.sqrt(n_inputs + n_neurons)
    weights = pd.DataFrame(truncnorm.rvs(-1, 1, size=(n_inputs, n_neurons), scale=stddev, random_state=42),
                           columns = [str(i) for i in range(1, n_neurons + 1)],
                           index=['w' + str(x) for x in range(1, n_inputs + 1)])
    
    return pd.DataFrame(weights)

In [157]:
weights = init_weights(X, 3)
weights

,1,2,3
w1,-0.136844,0.549885,0.258071
w2,0.107289,-0.396862,-0.396894
w3,-0.536119,0.426554,0.109988
w4,0.230176,-0.596733,0.580865
w5,0.381769,-0.325099,-0.363401
w6,-0.361386,-0.215993,0.026802
w7,-0.073822,-0.230984,0.121806


# 2. Forward Propagate

In [158]:
def crossenthropy(x):
    return 1. / (1. + np.exp(-x))

In [159]:
def neuron_layer(X, n_neurons, weights, activation=None):
    b = np.zeros(n_neurons)
    Z = np.dot(X, weights) + b
    if activation is not None:
        return activation(Z)
    else:
        return Z

In [163]:
pd.DataFrame(neuron_layer(X, 3, weights, activation=crossenthropy))

,0,1,2
0,0.608237,0.034629,0.727770
1,0.705093,0.048035,0.689999
2,0.557561,0.036001,0.671696
3,0.614428,0.031886,0.656537
4,0.669464,0.062339,0.725904
...,...,...,...
205,0.481503,0.016230,0.644419
206,0.427186,0.009575,0.639915
207,0.147280,0.007226,0.687458
208,0.485370,0.013116,0.643937


In [161]:
y

0      1
1      1
2      1
3      1
4      1
      ..
205    3
206    3
207    3
208    3
209    3
Name: target, Length: 210, dtype: int64

In [126]:
neuron_layer([1, 2], 3, weights.iloc[1])

array([0.47264925, 0.47264925, 0.47264925])

# 3. Back Propagate Error